In [59]:
import pandas as pd

# Path to your Excel file
excel_file = r'C:\Users\lopez\OneDrive\Desktop\WORKPROJECTS\DetalleManiobraProject\DETALLE MANIOBRAS HISTORICO.xlsx'

# Specify sheet names
sheet_names = ['1A-1D', '2A-2D', '3A-3D']

# Read all sheets into a dictionary of DataFrames
dfs = pd.read_excel(excel_file, sheet_name=sheet_names)
df_columnas = ['ASSET', 'PROYECTO', 'POZO', 'INTERVENCION_ID', 'FECHA', 'PROGRAMA_ID', 'TIPOPROGRAMA', 'FECHAINIPROG', 'FECHAFINPROG', 'NROPD', 'FECHAPD', 'HORAINIMANIONBRA', 'HORAFINMANIONBRA', 'MANIOBRA', 'MANDRIL', 'QPOZO', 'PPOZO', 'TIPO_MANIOBRA']
#maniobra_elements = ['Pesca Valvula con Resultado / Positivo', 'Coloca Valvula con Resultado / Positivo']
#TPrograma_elements = ['Regulación Válvulas / Liberación de válvulas']



def delta_qp(df):
    maniobra_elements = ['Pesca Valvula con Resultado / Positivo', 'Coloca Valvula con Resultado / Positivo']
    TPrograma_elements = ['Regulación Válvulas / Liberación de válvulas']
    df_filtered = df[df['MANIOBRA'].isin(maniobra_elements) & df['TIPOPROGRAMA'].isin(TPrograma_elements)]
    df_aggregated = df_filtered.groupby(['ASSET','PROYECTO','INTERVENCION_ID','PROGRAMA_ID','FECHA','POZO','FECHAINIPROG', 'FECHAFINPROG', 'MANDRIL', 'MANIOBRA'])[['QPOZO','PPOZO']].sum().reset_index()
    df_qpozo_pivot = df_aggregated.pivot_table(index=['ASSET','PROYECTO','INTERVENCION_ID','PROGRAMA_ID', 'FECHA', 'POZO', 'FECHAINIPROG', 'FECHAFINPROG','MANDRIL'], columns='MANIOBRA', values='QPOZO').reset_index()
    df_ppozo_pivot = df_aggregated.pivot_table(index=['ASSET','PROYECTO','INTERVENCION_ID','PROGRAMA_ID', 'FECHA', 'POZO', 'FECHAINIPROG', 'FECHAFINPROG','MANDRIL'], columns='MANIOBRA', values='PPOZO').reset_index()
    
    df_pivot  = pd.merge(df_qpozo_pivot, df_ppozo_pivot, on=['ASSET','PROYECTO','INTERVENCION_ID','PROGRAMA_ID', 'FECHA', 'POZO','FECHAINIPROG', 'FECHAFINPROG', 'MANDRIL'], suffixes=('_QPOZO', '_PPOZO'))
    
    df_pivot['DeltaQ'] = df_pivot['Pesca Valvula con Resultado / Positivo_QPOZO'] - df_pivot['Coloca Valvula con Resultado / Positivo_QPOZO']
    df_pivot['DeltaP'] = df_pivot['Coloca Valvula con Resultado / Positivo_PPOZO'] - df_pivot['Pesca Valvula con Resultado / Positivo_PPOZO']
    df_pivot = df_pivot.dropna(subset=['DeltaQ'])

    return df_pivot


processed_dfs = {sheet: delta_qp(df) for sheet, df in dfs.items()}

df_1AD = processed_dfs['1A-1D']
df_2D = processed_dfs['2A-2D']
df_3D = processed_dfs['3A-3D']

df_1AD.to_csv('1AD', index=False)
df_2D.to_csv('2AD', index=False)
df_3D.to_csv('3AD', index=False)




In [60]:
import pandas as pd
import plotly.express as px

# Assuming processed_dfs is a dictionary of dataframes loaded from your Excel file
# processed_dfs = {'Sheet1': df1, 'Sheet2': df2, ...}

def select_sheet_and_pozo():
    print("Select a sheet:")
    sheet_names = list(processed_dfs.keys())
    for i, sheet in enumerate(sheet_names):
        print(f"{i + 1}. {sheet}")

    while True:
        try:
            sheet_index = int(input("Enter the number corresponding to the sheet you want to use: ")) - 1
            if sheet_index not in range(len(sheet_names)):
                raise ValueError
            selected_sheet = sheet_names[sheet_index]
            break
        except ValueError:
            print("Invalid selection. Please enter a number corresponding to one of the listed sheets.")

    selected_pozo = input("Enter the POZO you want to use: ")
    return selected_sheet, selected_pozo


def plot_mandril_counts(selected_sheet, selected_pozo):
    # Filter dataframe for the selected POZO
    data = processed_dfs[selected_sheet]
    data_pozo = data[data['POZO'] == selected_pozo].copy()  # Ensure to work on a copy

    if data_pozo.empty:
        print(f"No data found for POZO: {selected_pozo}")
        return
    
    # Filter rows with empty DeltaQ
    data_pozo = data_pozo.dropna(subset=['DeltaQ'])

    # Count Mandrils
    mandril_counts = data_pozo.groupby('MANDRIL').size().reset_index(name='Count')

    # Plot Mandril counts
    fig = px.bar(mandril_counts, x='MANDRIL', y='Count', 
                 title=f'Mandril Counts for POZO: {selected_pozo}',
                 labels={'MANDRIL': 'Mandril #', 'Count': 'Count'})

    fig.update_layout(xaxis_title='Mandril #', yaxis_title='Count', showlegend=False)
    fig.show()


# Execution
selected_sheet, selected_pozo = select_sheet_and_pozo()
plot_mandril_counts(selected_sheet, selected_pozo)


Select a sheet:
1. 1A-1D
2. 2A-2D
3. 3A-3D


In [64]:
import pandas as pd
import plotly.graph_objects as go

# Assuming processed_dfs is defined as in your previous scripts
# processed_dfs = {'Sheet1': df1, 'Sheet2': df2, ...}

def select_mandril_to_plot(df, selected_pozo):
    mandril_counts = df[df['POZO'] == selected_pozo]['MANDRIL'].unique()

    while True:
        try:
            selected_mandril = int(input("Enter the MANDRIL number you want to plot: "))
            if selected_mandril not in mandril_counts:
                raise ValueError
            break
        except ValueError:
            print("Invalid selection. Please enter a valid MANDRIL number.")

    return selected_mandril


def aggregate_data_for_mandril(df, selected_pozo, selected_mandril):
    # Filter data for selected POZO and Mandril
    data_selected = df[(df['POZO'] == selected_pozo) & (df['MANDRIL'] == selected_mandril)].copy()

    if data_selected.empty:
        raise ValueError(f"No data found for MANDRIL {selected_mandril} in POZO {selected_pozo}")

    # Calculate DeltaQ and DeltaP
    data_selected['DeltaQ'] = data_selected['Pesca Valvula con Resultado / Positivo_QPOZO'] - data_selected['Coloca Valvula con Resultado / Positivo_QPOZO']
    data_selected['DeltaP'] = data_selected['Pesca Valvula con Resultado / Positivo_PPOZO'] - data_selected['Coloca Valvula con Resultado / Positivo_PPOZO']

    # Calculate averages
    avg_deltaq = data_selected['DeltaQ'].mean()
    avg_deltap = data_selected['DeltaP'].mean()

    # Sort data by FECHA (date)
    data_selected = data_selected.sort_values(by='FECHA')

    return data_selected, avg_deltaq, avg_deltap


def plot_histogram(selected_sheet, selected_pozo, selected_mandril):
    # Aggregate data for the selected POZO and Mandril
    data_selected, avg_deltaq, avg_deltap = aggregate_data_for_mandril(processed_dfs[selected_sheet], selected_pozo, selected_mandril)

    # Create figure for histogram
    fig = go.Figure()

    # Add bar for DeltaQ
    fig.add_trace(go.Bar(
        x=data_selected['FECHA'],
        y=data_selected['DeltaQ'],
        name='DeltaQ',
        marker_color='blue',
        opacity=0.75,
        hoverinfo='x+text',
        text=data_selected.apply(lambda row: f"FECHA: {row['FECHA'].strftime('%d/%m/%Y')}<br>FECHAINIPROG: {row['FECHAINIPROG']}<br>FECHAFINPROG: {row['FECHAFINPROG']}<br>PROGRAMA_ID: {row['PROGRAMA_ID']}<br>INTERVENCION_ID: {row['INTERVENCION_ID']}", axis=1)
    ))

    # Add bar for DeltaP
    fig.add_trace(go.Bar(
        x=data_selected['FECHA'],
        y=data_selected['DeltaP'],
        name='DeltaP',
        marker_color='orange',
        opacity=0.75,
        hoverinfo='x+text',
        text=data_selected.apply(lambda row: f"FECHA: {row['FECHA'].strftime('%d/%m/%Y')}<br>FECHAINIPROG: {row['FECHAINIPROG']}<br>FECHAFINPROG: {row['FECHAFINPROG']}<br>PROGRAMA_ID: {row['PROGRAMA_ID']}<br>INTERVENCION_ID: {row['INTERVENCION_ID']}", axis=1)
    ))

    fig.update_layout(
        title=f'Distribution of DeltaQ and DeltaP for MANDRIL {selected_mandril} in POZO {selected_pozo}',
        xaxis_title='FECHA',
        yaxis_title='Value',
        showlegend=True,
        height=600,
        barmode='group',  # Group bars for DeltaQ and DeltaP
        bargap=0.1,  # Gap between bars
        bargroupgap=0.1,  # Gap between groups of bars
        margin=dict(l=50, r=50, t=80, b=50),  # Adjust margins for better layout
        xaxis=dict(type='category')  # Use category axis for FECHA
    )

    # Add annotations for average DeltaQ and DeltaP
    fig.add_annotation(
        xref="paper", yref="paper",
        x=0.5, y=1.1,
        text=f"Avg DeltaQ: {avg_deltaq:.2f}",
        showarrow=False,
        align="left",
        bgcolor="white",
        bordercolor="black",
        borderwidth=1
    )

    fig.add_annotation(
        xref="paper", yref="paper",
        x=1.0, y=1.1,
        text=f"Avg DeltaP: {avg_deltap:.2f}",
        showarrow=False,
        align="left",
        bgcolor="white",
        bordercolor="black",
        borderwidth=1
    )

    fig.show()

# Example of usage:
#selected_sheet = 'Sheet1'  # Replace with your selected sheet
#selected_pozo = 'POZO1'    # Replace with your selected POZO
selected_mandril = select_mandril_to_plot(processed_dfs[selected_sheet], selected_pozo)

plot_histogram(selected_sheet, selected_pozo, selected_mandril)
